In [3]:
import os
import sys
sys.path.append('../..')

import pyedflib
import numpy as np
import os
import matplotlib.pyplot as plt
from scipy import signal
from imp import reload
import pandas as pd
import pywt
import readFiles
import databasePreparation
import preProcessing
import readEogertData
import math
from scipy import ndimage, misc
import json
import string
import neurokit2 as nk
import mne
import eeg_label

from visualization import plotEogElectrodesSignal, plotVertHorEOG, plotSaccadeSpanInSignal, plotTransformAndPeaks
from classificationUtils import peaksToBinary, peaksBinarySaccadeAnalysis, peaksBinaryToString, countEyeMovements, findPeaks
from evaluationUtils import countCorrectDetected, calculatePrecision, calculateRecall

In [5]:
# Reading the signals from the EDF file

edfFileName = 'C:/Users/Ricardo/source/enac-eog-analysis/src/labelAnnotation/EOGLabel-main/eegDataSet.edf'
eyesData = readFiles.readEog(edfFileName, [256, 257, 258, 259])

freqSample = 2048

ValueError: could not broadcast input array from shape (0,) into shape (1540096,)

In [ ]:
verticalEOG, horizontalEOG = preProcessing.electrodesToVertHori(eyesData, 0, 1, 2, 3)

verticalEogDenoised = ndimage.median_filter(verticalEOG, size = 200)
horizontalEogDenoised = ndimage.median_filter(horizontalEOG, size = 200)

verticalEOG = verticalEogDenoised
horizontalEOG = horizontalEogDenoised

verticalEOG3 = np.reshape(verticalEOG, (len(verticalEOG),1))
horizontalEOG3 = np.reshape(horizontalEOG, (len(horizontalEOG),1))

coefVertMexHat, freqsVertMexHat = pywt.cwt(verticalEOG, 30, 'mexh')
coefHoriMexHat, freqsHoriMexHat = pywt.cwt(horizontalEOG, 30, 'mexh')

coefVertMexHat1 = np.reshape(coefVertMexHat, (len(coefVertMexHat[0]),1))
coefHoriMexHat1 = np.reshape(coefHoriMexHat, (len(coefHoriMexHat[0]),1))

#coefVertMexHat2 = preProcessing.zeroPadArtifacts(coefVertMexHat1, 70)

# Peakfinding

# Flatten the vectors so they can be used in the signal.find_peaks function
vertCwt = coefVertMexHat1.flatten()
horiCwt = coefHoriMexHat1.flatten()

# Thresholds of the saccade peaks
peaksHeight = np.array([90, 600])
minSaccadeDistance = 100

# Returns peaks' indexes and their properties in a dict
# The peaks are detected in the module version of signal in order to find the negative peaks as well
peaksV = findPeaks(vertCwt, peaksHeight[0], peaksHeight[1], minSaccadeDistance)
peaksH = findPeaks(horiCwt, peaksHeight[0], peaksHeight[1], minSaccadeDistance)

vertPeaksBinary = peaksToBinary(peaksV, vertCwt)
horiPeaksBinary = peaksToBinary(peaksH, horiCwt)

upSaccadeStartEnd, downSaccadeStartEnd = peaksBinarySaccadeAnalysis(vertPeaksBinary, jumpIntervalThreshold=500, saccadeLowerThreshold=500, saccadeUpperThreshold=8000)
print(upSaccadeStartEnd,downSaccadeStartEnd)
leftSaccadeStartEnd, rightSaccadeStartEnd = peaksBinarySaccadeAnalysis(horiPeaksBinary, jumpIntervalThreshold=500, saccadeLowerThreshold=500, saccadeUpperThreshold=8000)
print('Left Saccades', leftSaccadeStartEnd, '\n', 'Right Saccade')

# Detecting saccades candidates and blink candidates
vertPeaksString = peaksBinaryToString(vertPeaksBinary[2])
horiPeaksString = peaksBinaryToString(horiPeaksBinary[2])

up, down, left, right, blinks = countEyeMovements(vertPeaksString, horiPeaksString)

fileName = "Testdata0602"
partName = "{}".format(sectionName)
plotName = "{}_{}".format(fileName, partName)

infoDict = {"name": plotName,
            "wavelet": "Mexican Hat",
            "scale": "30",
            "lowThreshold": str(peaksHeight[0]),
            "highThreshold": str(peaksHeight[1]),
            "upSaccadeCount": str(up),
            "downSaccadeCount": str(down),
            "leftSaccadeCount": str(left),
            "rightSaccadeCount": str(right),
            "blinkCount": str(blinks)}

infoJson = json.dumps(infoDict, indent=3)

with open("{}{}_info.json".format(resultsPath, plotName), "w") as outfile: 
    outfile.write(infoJson) 

print('Part Name: {}\n'.format(plotName))
print('Wavelet: Mexican Hat \nScale: 30\nPeak Threshold: {}'.format(peaksHeight))
print('\n(\'0110\') Upward Saccade Count: {}\n(\'1001\') Downward Saccade Count: {}\n(\'0110\') Left Saccade Count: {}\n(\'1001\') Right Saccade Count: {}\n(\'010\') Blink Count: {}'.format(up,down,left,right,blinks))

# All the plots

plotVertHorEOG(verticalEogDenoised, horizontalEogDenoised,  mode='both')

plt.savefig('{}{}_eog.png'.format(resultsPath, plotName))
plt.show()

plotSaccadeSpanInSignal(verticalEOG, upSaccadeStartEnd, downSaccadeStartEnd)
plt.show()

plotTransformAndPeaks(coefVertMexHat1, peaksV, verticalEOG, 'Vertical')
plt.savefig('{}{}_eogVTransform.png'.format(resultsPath,plotName))
plt.show()

plotSaccadeSpanInSignal(horizontalEOG, leftSaccadeStartEnd, rightSaccadeStartEnd)
plt.show()

plotTransformAndPeaks(coefHoriMexHat1, peaksH, horizontalEOG, 'Horizontal')
plt.savefig('{}{}_eogHTransform.png'.format(resultsPath, plotName))
plt.show() 

# Saving the parts of the signal into .matfiles
#databasePreparation.arrayToMat(part, plotName)